In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

C:\Users\Abhishek\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [56]:
# load training and test data
train = pd.read_csv('data/train_processed.csv')
test = pd.read_csv('data/test_processed.csv')

In [57]:
# preprocess skills feature

# determine unique skills
skills = set()
for m in train.skills:
    skills.update(g for g in m.split('|'))
skills = sorted(skills)

#make a column for each skill
for skill in skills:
    train[skill] = [int(skill in s.split('|')) for s in train.skills]

In [58]:
# do same calculation for test set as well
for skill in skills:
    test[skill] = [int(skill in s.split('|')) for s in test.skills]

In [59]:
# determine problem tags
problem_tags = set()
for tag1, tag2, tag3, tag4, tag5 in zip(train.tag1, train.tag2, train.tag3, train.tag4, train.tag5):
    problem_tags.update([tag1, tag2, tag3, tag4, tag5])
problem_tags = sorted(problem_tags)

In [60]:
problem_tags[0] = 'Not Specified'

In [61]:
# make a column for each problem type
for problem_tag in problem_tags:
    train[problem_tag] = [int(problem_tag in tags) for tags in train[['tag1', 'tag2', 'tag3', 'tag4', 'tag5']].values]

In [62]:
for problem_tag in problem_tags:
    test[problem_tag] = [int(problem_tag in tags) for tags in test[['tag1', 'tag2', 'tag3', 'tag4', 'tag5']].values]

In [63]:
# balanced dataset
%run scripts/helper.py
train = balanced_sample(train)

In [64]:
# response variable
y = train.solved_status

## Prepare a very simple model based on just the accuracy of the problem

In [65]:
%run scripts/features.py
%run scripts/models.py
%run scripts/eval.py

In [66]:
model = build_extreme_gradient_boosting(train, test)
# model2 = build_xgboost_model(train, test)
# model2 = build_random_forest_classifier(train, test)
# model3 = build_logistic_regression_model(train, test)
# model = build_knn_classifier(train, test)
# model2 = build_extra_trees_classifier(train, test)

In [67]:
scores = eval_models([model], train, y)

accuracy score: 0.809285
combined score: 0.809285
accuracy score: 0.807453
combined score: 0.807453
accuracy score: 0.802516
combined score: 0.802516
accuracy score: 0.803711
combined score: 0.803711
accuracy score: 0.800366
combined score: 0.800366



In [68]:
print 'Mean: %f and Standard Deviation: %f' %(np.mean(scores), np.std(scores))

Mean: 0.804666 and Standard Deviation: 0.003260


In [ ]:
predsTrain, predsTest, y_train, y_test = analyze_output(train)

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, predsTest)

In [ ]:
misclassified = train.ix[(y_test != predsTest).index.values]
misclassified['preds'] = predsTest

In [ ]:
misclassified_class_0 = misclassified[misclassified.solved_status == 0]

In [ ]:
misclassified_class_0[['accuracy', 'solved_status', 'preds']]

In [ ]:
sns.pointplot(x="solved_status", y="accuracy", data=misclassified);

In [69]:
# train model
import time

start_time = time.time()

model.fit(train, y)

elapsed_time = time.time() - start_time

print 'It took %f seconds to train the model ' %(elapsed_time)

It took 37.419000 seconds to train the model 


In [70]:
# predictions1  = model1.predict(test)
# predictions2 = model2.predict(test)

# ensemble_preds = np.array(majority_voting([predictions1, predictions2]))
predictions = model.predict(test)

In [71]:
# submissions
submission_df = pd.DataFrame({'Id': test.id.values, 'solved_status': predictions.astype(int)})
submission_df.to_csv('submissions/thirtieth_xgb_one_hot_encoded_skills.csv', index=False)